In [2]:
import pickle
from nltk.sentiment.util import *
from textblob import TextBlob
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [6]:
source_list = ["EconomicTimes", "Hindu", "HindustanTimes", "IndianExpress", "TimesOfIndia"]
REPLACE_NO_SPACE = re.compile("(\')|(\,)|(\")|(\()|(\))|(\[)|(\])")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\/)|(<p>)|(\\n)|(\;)|(\:)")
REPLACE_WITH_PERIOD = re.compile("(\-)|(\!)|(\?)")

In [9]:
articlesLST = []
issues = ['Ayodhya', 'TripleTalaq']
def conductAnalysis():
    for issue in issues:
        print(issue)
        for source in source_list:
            filename = "data/" + issue + '/' + source
            file = open(filename, 'rb')
            articles = pickle.load(file)
            print(source + " " + str(len(articles)))

            articles = [REPLACE_NO_SPACE.sub("", article) for article in articles]
            articles = [REPLACE_WITH_SPACE.sub(" ", article) for article in articles]
            articles = [REPLACE_WITH_PERIOD.sub(".", article) for article in articles]

            analysis = []
            count = [0, 0]
            for article in articles:
                thing = TextBlob(article)
                if thing.sentiment[0] > 0.1:
                    count[0] += 1
                else:
                    count[1] += 1
                analysis.append({'polarity': thing.sentiment[0], 'subjectivity': thing.sentiment[1]})
                articlesLST.append([source, article, thing.sentiment[0], thing.sentiment[1]])
    #         print(count)

            filename1 = filename + '_clean'
            file = open(filename1, 'wb')
            pickle.dump(articles, file)
            file.close()
            filename2 = filename + '_scores'
            file = open(filename2, 'wb')
            pickle.dump(analysis, file)
        articlesDF = pd.DataFrame(articlesLST,columns=['source', 'article', 'polarity', 'subjectivity'])
        file1 = issue + 'Articles Dataframe'
        file = open(file1, 'wb')
        pickle.dump(articlesDF, file)

In [10]:
# uses textblob to generate sentiments
conductAnalysis()

Ayodhya
EconomicTimes 69
Hindu 124
HindustanTimes 83
IndianExpress 121
TimesOfIndia 208
TripleTalaq
EconomicTimes 88
Hindu 242
HindustanTimes 339
IndianExpress 366
TimesOfIndia 693


In [11]:
# Read from pickle
file = open("TripleTalaqArticles Dataframe", 'rb')
articlesDF_tt = pickle.load(file)
file = open("AyodhyaArticles Dataframe", 'rb')
articlesDF_ay = pickle.load(file)

In [12]:
# tokenize sentences into POS
# find sentences that contain certain words related to BJP, or Ayodhya/talaq

import nltk
from nltk.tokenize import word_tokenize

sentList = []
bjp = ["Narendra Modi", "Prime Minister", "BJP", "Bharatiya Janata Party"]
ayodhya = ["Ayodhya", "mandir", "Supreme Court", "Uttar Pradesh", "U.P.", 'Ramjanmabhoomi', 
           'Babri', 'Masjid', 'masjid', 'akhara', 'hindu', 'waqf', 'Hindu', 'Waqf', 'Akhara']
talaq = ['triple', 'Triple', 'Talaq', 'talaq', 'Shayara', 'Bano', 'Muslim Personal', 'AIMPLB', 'instant']

In [21]:
for index, row in articlesDF_ay.iterrows():
    sentences = row['article'].split('.')
    for sentence in sentences:
        text = word_tokenize(sentence)
        if (not set(bjp).isdisjoint(text)) or (not set(ayodhya).isdisjoint(text)):
            sentMap = {'source': row['source'], 'sentence': sentence, 'score': ''}
            sentList.append(sentMap)

In [22]:
sentencesDF = pd.DataFrame(sentList,columns=['source', 'sentence', 'score'])
print(len(sentencesDF))

file = open('Ayodhya_Sentences', 'wb')
pickle.dump(sentencesDF, file)

21875


In [23]:
for index, row in articlesDF_tt.iterrows():
    sentences = row['article'].split('.')
    for sentence in sentences:
        text = word_tokenize(sentence)
        if (not set(bjp).isdisjoint(text)) or (not set(talaq).isdisjoint(text)):
            sentMap = {'source': row['source'], 'sentence': sentence, 'score': ''}
            sentList.append(sentMap)

In [24]:
sentencesDF = pd.DataFrame(sentList,columns=['source', 'sentence', 'score'])
print(len(sentencesDF))

file = open('TripleTalaq_Sentences', 'wb')
pickle.dump(sentencesDF, file)

34128


In [25]:
# Read from file
file = open('TripleTalaq_Sentences', 'rb')
sentencesDF_tt = pickle.load(file)

file = open('Ayodhya_Sentences', 'rb')
sentencesDF_ay = pickle.load(file)

In [26]:
# Store random 20% of each publication's sentences into a CSV
pubCount = sentencesDF_tt['source'].value_counts()
train_tt, test_tt = train_test_split(sentencesDF_tt, test_size = 0.8, stratify=sentencesDF_tt.source)

pubCount = sentencesDF_ay['source'].value_counts()
train_ay, test_ay = train_test_split(sentencesDF_ay, test_size = 0.8, stratify=sentencesDF_ay.source)

In [26]:
file = open('TripleTalaq_Train', 'wb')
pickle.dump(train_tt, file)
train_tt.to_csv('TripleTalaq_train.csv')

file = open('TripleTalaq_Test', 'wb')
pickle.dump(test_tt, file, protocol=2)

file = open('Ayodhya_Train', 'wb')
pickle.dump(train_ay, file)
train_ay.to_csv('TripleTalaq_train.csv')

file = open('Ayodhya_Test', 'wb')
pickle.dump(test_ay, file, protocol=2)

### Start From Here when running analysis
Everything above this is pre-processing

In [1]:
# read from file
train_ay = pd.read_csv('Ayodhya_train_annotated.csv')
file = open('Ayodhya_Test', 'rb')
test_ay = pickle.load(file)

train_tt = pd.read_csv('Ayodhya_train_annotated.csv')
file = open('TripleTalaq_Sentences', 'rb')
test_tt = pickle.load(file)

NameError: name 'pd' is not defined

In [42]:
import random
 
sentiment_data = zip(train["sentence"], train["score"]) 
train_X, train_y = zip(*sentiment_data)

test_data = zip(test['sentence'], test['score'])
test_X, test_y = zip(*test_data)

In [43]:
from nltk import word_tokenize
from nltk.sentiment.util import mark_negation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.base import TransformerMixin

In [44]:
unigram_bigram_clf = Pipeline([
    ('vectorizer', CountVectorizer(analyzer="word",
                                   ngram_range=(1, 2),
                                   tokenizer=word_tokenize,
                                   # tokenizer=lambda text: mark_negation(word_tokenize(text)),
                                   preprocessor=lambda text: text.replace("<br />", " "),)),
    ('classifier', LinearSVC())
])

In [45]:
unigram_bigram_clf.fit(train_X, train_y)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2),
        preprocessor=<function <lambda> at ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [46]:
results = unigram_bigram_clf.predict(test_X)

In [47]:
test['score'] = results.tolist()

In [48]:
file = open('TripleTalaq_test_results', 'wb')
pickle.dump(test, file)
test.to_csv('TripleTalaqtest_results.csv')

In [50]:
# read from file
file = open('TripleTalaq_test_results', 'rb')
test_res = pickle.load(file)

In [51]:
test_res.head()

,source,sentence,score
0,EconomicTimes,Politics and Nation Triple,0
1,EconomicTimes,talaq supporters collect ‘5 lakh signatures’ ...,0
2,EconomicTimes,Making this claim Jalsa Tehaffuz Personal Law ...,0
3,EconomicTimes,“Article 25 of the Constitution gives equal op...,-1
4,EconomicTimes,The unilateral triple talaq which Muslims a...,0


In [53]:
test_res.groupby(['source', 'score']).size()

source          score
EconomicTimes   -1         58
                 0        476
                 1         35
Hindu           -1        157
                 0        946
                 1         85
HindustanTimes  -1        138
                 0       1720
                 1        130
IndianExpress   -1        205
                 0       1707
                 1        143
TimesOfIndia    -1        341
                 0       2766
                 1        255
dtype: int64

In [54]:
test_res.groupby(['source']).sum()

,score
source,
EconomicTimes,-23
Hindu,-72
HindustanTimes,-8
IndianExpress,-62
TimesOfIndia,-86


# Accuracy: 0.633